# Polars Run

In [7]:
import os
import polars as pl
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [8]:
# PATH_DATA = '../../df-showdown-data/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output/'

SLA_NAME = 'polars-sla.pkl'
MEM_NAME = 'polars-mem.pkl'


In [9]:
all_sla = {}
all_memory = {}

In [10]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

##     Task1 & Task2- I/O

In [20]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [21]:
for i in range(10):
    print(i)
    if i<=5:
        fn = 'data_{}.csv'.format(i)
        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1 = pl.read_csv(PATH_DATA+fn)
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        nrows = str(int(dtemp1.shape[0]/1000000))
        all_sla['task1']['csv'][nrows]=end
        all_memory['task1']['csv'][nrows]=end_mem

        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1.write_csv(TEMP_PATH + 'temp.csv')
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        all_sla['task2']['csv'][nrows]=end
        all_memory['task2']['csv'][nrows]=end_mem

        del dtemp1
        time.sleep(3)
        gc.collect()
        os.remove(TEMP_PATH + 'temp.csv')
        time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = pl.read_parquet(PATH_DATA+fn)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.write_parquet(TEMP_PATH + 'temp.parquet')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    time.sleep(3)
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1
2
3
4
5
6
7
8
9


In [22]:
all_sla

{'task1': {'csv': {'0': 0.030919900000014877,
   '1': 0.3084856999999488,
   '5': 1.0609346999999616,
   '10': 2.0551040000000285,
   '25': 5.5099544000000265,
   '50': 11.726973900000019},
  'prq': {'0': 0.03822700000000623,
   '1': 0.17389030000003913,
   '5': 0.6843605999999909,
   '10': 0.7158807000000138,
   '25': 1.5400334999999927,
   '50': 2.9179092000000537,
   '75': 4.4800702,
   '100': 5.842930600000045,
   '130': 7.630183899999906,
   '175': 10.953879299999926}},
 'task2': {'csv': {'0': 0.05873319999994919,
   '1': 0.34670369999997774,
   '5': 1.4920029000000454,
   '10': 3.0747118999999543,
   '25': 7.094185200000027,
   '50': 11.06910920000007},
  'prq': {'0': 0.12076949999999442,
   '1': 0.5575091000000043,
   '5': 2.53777869999999,
   '10': 5.172184300000026,
   '25': 10.350711700000033,
   '50': 17.822521400000028,
   '75': 27.643802700000037,
   '100': 35.53102279999996,
   '130': 46.54294879999998,
   '175': 62.06326110000009}}}

In [23]:
all_memory

{'task1': {'csv': {'0': 0.028782591999998886,
   '1': 0.24958156799999998,
   '5': 0.953692160000001,
   '10': 1.378066431999999,
   '25': 2.2250496,
   '50': 3.8796042240000013},
  'prq': {'0': -0.008777728000000096,
   '1': -0.02224947200000038,
   '5': 0.19870924800000012,
   '10': 0.8999854079999992,
   '25': 1.7124843520000006,
   '50': 3.057057792,
   '75': 3.5248373760000007,
   '100': 3.9399792639999998,
   '130': 4.993519616000002,
   '175': 6.080352255999998}},
 'task2': {'csv': {'0': 0.01021952000000148,
   '1': -0.007311360000000988,
   '5': -0.018190335999999974,
   '10': -0.07151615999999983,
   '25': -0.208650239999999,
   '50': -0.6279741439999995},
  'prq': {'0': -0.013377536000000134,
   '1': 6.143999999963512e-05,
   '5': -0.008433664000000007,
   '10': -0.14562508799999918,
   '25': -0.13502054399999963,
   '50': -0.01539686399999951,
   '75': 0.027217920000000007,
   '100': 0.07061913600000125,
   '130': -0.42483302400000156,
   '175': -0.6214696959999984}}}

In [24]:
# del dtemp1
# gc.collect()

## Get Data for other task

In [26]:
all_df = [pl.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [4,5,6,7]]
df_right = pl.read_parquet(PATH_DATA+'data_to_join.parquet')

In [27]:
len(all_df)

4

## Task3 - Sorting

In [28]:
n_task = 'task3'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [33]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.sort(['Date','Amount'],ascending=[True,False])
    dtemp2 = dtemp1.with_columns(
            pl.col('Amount').sort(descending=True),
            pl.col('Date').sort(descending=False),
    )
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()


25.02
50.04
75.06
100.08


In [34]:
all_sla[n_task]

{'25': 0.9957250999996177,
 '50': 3.3215760000002774,
 '75': 3.8688915000002453,
 '100': 5.026626400000168}

In [35]:
all_memory[n_task]

{'25': 0.6112706560000003,
 '50': 1.6995368959999997,
 '75': 2.9167083520000006,
 '100': 3.0360657920000005}

## Task4 - Filtering

In [38]:
n_task = 'task4'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [40]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))]
    dtemp2 = (dtemp1
                .filter((pl.col('Amount')>300) & (pl.col('Status').is_in(filt1)))
              )
    
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()
    


25.02
50.04
75.06
100.08


In [41]:
all_sla[n_task]

{'25': 2.4028910000001815,
 '50': 5.02974630000017,
 '75': 10.887932299999648,
 '100': 16.885443499999838}

In [42]:
all_memory[n_task]

{'25': 2.150805503999999,
 '50': 3.068329984,
 '75': 5.125693439999999,
 '100': 4.038283264}

## Task5 - Merging

In [1]:
n_task = 'task5'
all_sla[n_task] = {}
all_memory[n_task] = {}

NameError: name 'all_sla' is not defined

In [56]:
df_right = df_right.with_columns((pl.col('Date')+'_'+pl.col('ship-service-level')).alias('key_merge'))

In [57]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    dtemp1 = dtemp1.with_columns((pl.col('Date')+'_'+pl.col('ship-service-level')).alias('key_merge'))

    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.join(df_right,on='key_merge',how='left')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02


: 

: 

In [ ]:
all_sla[n_task]

{}

In [ ]:
all_memory[n_task]

{'25': 1.225998336,
 '50': 2.3351336959999998,
 '75': 3.5374899200000023,
 '100': 3.2817152000000007}

## Task6 - udf apply

In [31]:
n_task = 'task6'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [32]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [33]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06


MemoryError: Unable to allocate 573. MiB for an array with shape (75063450,) and data type uint64

In [34]:
all_sla[n_task]

{'25': 112.1799453000001, '50': 225.9367654}

In [35]:
all_memory[n_task]

{'25': 0.5384683519999989, '50': -2.4369602560000008}

## Task7 - aggregation

In [12]:
n_task = 'task7'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [13]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [14]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':[np.mean, np.size, p25, p75]})
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':'mean'})

    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

25.02
50.04
75.06
100.08


In [17]:
all_sla[n_task]

{'25': 3.367476599999975,
 '50': 7.263250599999992,
 '75': 9.238950799999998,
 '100': 14.18757690000001}

In [18]:
all_memory[n_task]

{'25': 0.15176499200000038,
 '50': -0.06929203199999989,
 '75': 0.05231001599999985,
 '100': -0.5718097919999998}

## Save results

In [19]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output/pandas-mem.pkl']